In [ ]:
# train_gdb_top9.py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

# ----------------------------
# File paths & parameters
# ----------------------------
DATA_FILE = "Data.xlsx"                          # Original dataset (first column: Hv)
RANKING_CSV = "Selected_Feature_Ranking.csv"    # Primary source (should include 'feature' or similar column)
IMP_CSV = "Feature_Importance_GDB.csv"          # Secondary source
PCC_XLSX = "Selected_Features_PCC.xlsx"         # Fallback source (columns are remaining features)

TOP_K = 9
TEST_SIZE = 0.2
RANDOM_STATE = 42
CV_FOLDS = 5

OUT_MODEL = "GDB_top9_model.pkl"
OUT_METRICS = "GDB_top9_metrics.csv"
OUT_PREDICTIONS = "GDB_top9_predictions.csv"
OUT_PARITY = "GDB_top9_parity.png"
OUT_R2BAR = "GDB_top9_r2bar.png"

# ----------------------------
# 1) Load data
# ----------------------------
if not os.path.exists(DATA_FILE):
    raise FileNotFoundError(f"❌ File not found: {DATA_FILE}. Please ensure it is in the same directory as the script.")

df = pd.read_excel(DATA_FILE)

if "Hv" not in df.columns:
    raise ValueError(f"❌ Target column 'Hv' not found. Please make sure the target column is named 'Hv'. "
                     f"Current columns: {list(df.columns)}")

X_all = df.drop(columns=["Hv"])
y_all = df["Hv"].values

# ----------------------------
# 2) Select top 9 features (multiple sources)
# ----------------------------
selected_features = None

# Priority 1: Selected_Feature_Ranking.csv
if os.path.exists(RANKING_CSV):
    try:
        tmp = pd.read_csv(RANKING_CSV, encoding="utf-8-sig")
        feature_col_candidates = [c for c in tmp.columns if c.lower() in ("feature", "feature_name", "name")]
        feature_col = feature_col_candidates[0] if feature_col_candidates else tmp.columns[0]
        top_list = tmp[feature_col].astype(str).tolist()
        selected_features = [f for f in top_list if f in X_all.columns][:TOP_K]
        print(f"Loaded {len(selected_features)} matching features from {RANKING_CSV} (top {TOP_K} used).")
    except Exception as e:
        print("⚠️ Failed to read Selected_Feature_Ranking.csv, trying next source.", e)

# Priority 2: Feature_Importance_GDB.csv
if (selected_features is None or len(selected_features) < TOP_K) and os.path.exists(IMP_CSV):
    try:
        tmp = pd.read_csv(IMP_CSV, encoding="utf-8-sig")
        feat_cands = [c for c in tmp.columns if c.lower() in ("feature", "index", "unnamed: 0")]
        imp_cands = [c for c in tmp.columns if c.lower() in ("mean_importance", "importance", "score", "mean_score")]
        feat_col = feat_cands[0] if feat_cands else tmp.columns[0]
        imp_col = imp_cands[0] if imp_cands else (tmp.columns[1] if tmp.shape[1] > 1 else tmp.columns[0])
        tmp2 = tmp[[feat_col, imp_col]].rename(columns={feat_col: "feature", imp_col: "importance"})
        tmp2["feature"] = tmp2["feature"].astype(str)
        tmp2 = tmp2.sort_values("importance", ascending=False)
        ranked = [f for f in tmp2["feature"].tolist() if f in X_all.columns]
        if selected_features is None:
            selected_features = ranked[:TOP_K]
        else:
            for f in ranked:
                if f not in selected_features and len(selected_features) < TOP_K:
                    selected_features.append(f)
        print(f"After adding from {IMP_CSV}, total selected features: {len(selected_features)} (target {TOP_K}).")
    except Exception as e:
        print("⚠️ Failed to read Feature_Importance_GDB.csv, trying next source.", e)

# Fallback: Selected_Features_PCC.xlsx
if (selected_features is None or len(selected_features) < TOP_K) and os.path.exists(PCC_XLSX):
    try:
        tmp = pd.read_excel(PCC_XLSX, index_col=0)
        cols = [c for c in tmp.columns if c in X_all.columns]
        if selected_features is None:
            selected_features = cols[:TOP_K]
        else:
            for c in cols:
                if c not in selected_features and len(selected_features) < TOP_K:
                    selected_features.append(c)
        print(f"After adding from {PCC_XLSX}, total selected features: {len(selected_features)} (target {TOP_K}).")
    except Exception as e:
        print("⚠️ Failed to read Selected_Features_PCC.xlsx.", e)

# Final fallback: use the first TOP_K columns from X_all
if selected_features is None or len(selected_features) < TOP_K:
    print("⚠️ None of the above sources provided enough features. "
          "Using the first TOP_K columns (excluding Hv).")
    selected_features = list(X_all.columns[:TOP_K])

# Check final feature count
if len(selected_features) < TOP_K:
    raise ValueError(f"Final number of selected features < {TOP_K} (current: {len(selected_features)}). "
                     "Please check naming consistency.")

print("\nFinal 9 features used for modeling:")
for i, f in enumerate(selected_features, 1):
    print(f"{i}. {f}")

# ----------------------------
# 3) Train-test split
# ----------------------------
X = X_all[selected_features].to_numpy()
y = y_all

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=True
)

print(f"\nTraining samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

# ----------------------------
# 4) Define model & GridSearchCV
# ----------------------------
gbr = GradientBoostingRegressor(random_state=RANDOM_STATE)

param_grid = {
    "n_estimators": [100, 200, 400],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2, 3, 4, 6],
    "subsample": [1.0, 0.8, 0.6]
}

grid = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    cv=CV_FOLDS,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    verbose=1,
    refit=True
)

print("\nStarting GridSearchCV... (this may take a while depending on data size and grid range)")
grid.fit(X_train, y_train)

print("\n✅ GridSearch completed. Best parameters:")
print(grid.best_params_)
best_model = grid.best_estimator_

# ----------------------------
# 5) Evaluate model
# ----------------------------
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

metrics = {
    "r2_train": r2_train,
    "r2_test": r2_test,
    "rmse_train": rmse_train,
    "rmse_test": rmse_test,
    "best_params": [grid.best_params_]
}
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv(OUT_METRICS, index=False, encoding="utf-8-sig")
print(f"\n✅ Metrics saved to: {OUT_METRICS}")
print(metrics_df.T)

# ----------------------------
# 6) Save predictions
# ----------------------------
pred_df_train = pd.DataFrame({"set": "train", "y_true": y_train, "y_pred": y_train_pred})
pred_df_test = pd.DataFrame({"set": "test", "y_true": y_test, "y_pred": y_test_pred})
pd.concat([pred_df_train, pred_df_test], ignore_index=True).to_csv(OUT_PREDICTIONS, index=False, encoding="utf-8-sig")
print(f"✅ Predictions saved to: {OUT_PREDICTIONS}")

# ----------------------------
# 7) Parity plot
# ----------------------------
plt.figure(figsize=(7, 7))
plt.scatter(y_train, y_train_pred, label=f"Train (R²={r2_train:.3f}, RMSE={rmse_train:.3f})", alpha=0.7, s=40)
plt.scatter(y_test, y_test_pred, label=f"Test (R²={r2_test:.3f}, RMSE={rmse_test:.3f})", alpha=0.9, s=60, marker='X')

lims = [min(y.min(), y_train.min(), y_test.min(), y_train_pred.min(), y_test_pred.min()),
        max(y.max(), y_train.max(), y_test.max(), y_train_pred.max(), y_test_pred.max())]
plt.plot(lims, lims, 'k--', linewidth=1)
plt.xlim(lims)
plt.ylim(lims)
plt.xlabel("True Hv")
plt.ylabel("Predicted Hv")
plt.title("Parity Plot — Gradient Boosting (Top 9 Features)")
plt.legend()
plt.tight_layout()
plt.savefig(OUT_PARITY, dpi=300)
plt.show()
print(f"✅ Parity plot saved as: {OUT_PARITY}")

# ----------------------------
# 8) Bar chart of R² (train vs test)
# ----------------------------
plt.figure(figsize=(5, 5))
labels = ["Train", "Test"]
r2_vals = [r2_train, r2_test]
bars = plt.bar(labels, r2_vals, capsize=6)
plt.ylim(0, max(1.0, max(r2_vals) * 1.1))
plt.ylabel("R²")
plt.title("R² on Train and Test")
for bar, v in zip(bars, r2_vals):
    plt.text(bar.get_x() + bar.get_width() / 2, v + 0.02, f"{v:.3f}", ha='center', va='bottom')

plt.tight_layout()
plt.savefig(OUT_R2BAR, dpi=300)
plt.show()
print(f"✅ R² bar chart saved as: {OUT_R2BAR}")

# ----------------------------
# 9) Save model
# ----------------------------
joblib.dump(grid.best_estimator_, OUT_MODEL)
print(f"✅ Model saved as: {OUT_MODEL}")

# ----------------------------
# 10) Save top 9 feature list
# ----------------------------
pd.DataFrame({"top_features": selected_features}).to_csv("GDB_top9_features.csv", index=False, encoding="utf-8-sig")
print("✅ Top 9 feature list saved as: GDB_top9_features.csv")

print("\nAll tasks completed successfully ✅")